# Image viewer

This notebook is for inspecting timelapse microscopy data, with associated sinhgle-cell labels and tracks, showing the infection of human macrophages with Mycobacterium Tuberculosis (Mtb), acquired on an Opera Phenix confocal microscope. 

In [17]:
import napari
import os, glob
from macrohet import dataio, tile, visualise, notify
import numpy as np
from macrohet import visualise
import os
import re
import numpy as np
import cv2
import btrack
import zarr

### Load experiment of choice

The Opera Phenix is a high-throughput confocal microscope that acquires very large 5-dimensional (TCZXY) images over several fields of view in any one experiment. Therefore, a lazy-loading approach is chosen to mosaic, view and annotate these images. This approach depends upon Dask and DaskFusion. The first step is to load the main metadata file (typically called `Index.idx.xml` and located in the main `Images` directory) that contains the image filenames and associated TCXZY information used to organise the images.

In [16]:
import pandas as pd
df = pd.read_pickle('/mnt/SYNO/macrohet_syno/results/dfs/trimmed_df.pkl')

EOFError: Ran out of input

In [18]:
%%time
expt_ID = 'PS0000'

base_dir = f'/mnt/SYNO/macrohet_syno/data/{expt_ID}/'
# base_dir = f'/mnt/DATA/macrohet/{expt_ID}/'

metadata_fn = glob.glob(os.path.join(base_dir, 'acquisition/Images/Index*xml'))[0]
metadata = dataio.read_harmony_metadata(metadata_fn)  
metadata

Reading metadata XML file...


0it [00:00, ?it/s]

Extracting metadata complete!
CPU times: user 11.8 s, sys: 397 ms, total: 12.2 s
Wall time: 12.7 s


,id,State,URL,Row,Col,FieldID,PlaneID,TimepointID,ChannelID,FlimID,...,PositionZ,AbsPositionZ,MeasurementTimeOffset,AbsTime,MainExcitationWavelength,MainEmissionWavelength,ObjectiveMagnification,ObjectiveNA,ExposureTime,OrientationMatrix
0,0303K1F1P1R1,Ok,r03c03f01p01-ch1sk1fk1fl1.tiff,3,3,1,1,0,1,1,...,0,0.135583505,0,2021-04-16T19:09:33.84+01:00,488,522,40,1.1,0.1,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."
1,0303K1F1P1R2,Ok,r03c03f01p01-ch2sk1fk1fl1.tiff,3,3,1,1,0,2,1,...,0,0.135583505,0,2021-04-16T19:09:33.84+01:00,640,706,40,1.1,0.2,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."
2,0303K1F1P2R1,Ok,r03c03f01p02-ch1sk1fk1fl1.tiff,3,3,1,2,0,1,1,...,2E-06,0.135585502,0,2021-04-16T19:09:34.12+01:00,488,522,40,1.1,0.1,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."
3,0303K1F1P2R2,Ok,r03c03f01p02-ch2sk1fk1fl1.tiff,3,3,1,2,0,2,1,...,2E-06,0.135585502,0,2021-04-16T19:09:34.12+01:00,640,706,40,1.1,0.2,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."
4,0303K1F1P3R1,Ok,r03c03f01p03-ch1sk1fk1fl1.tiff,3,3,1,3,0,1,1,...,4E-06,0.135587499,0,2021-04-16T19:09:34.4+01:00,488,522,40,1.1,0.1,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113395,0609K75F9P1R2,Ok,r06c09f09p01-ch2sk75fk1fl1.tiff,6,9,9,1,74,2,1,...,0,0.135533601,266399.61,2021-04-19T21:14:19.477+01:00,640,706,40,1.1,0.2,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."
113396,0609K75F9P2R1,Ok,r06c09f09p02-ch1sk75fk1fl1.tiff,6,9,9,2,74,1,1,...,2E-06,0.135535598,266399.61,2021-04-19T21:14:19.757+01:00,488,522,40,1.1,0.1,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."
113397,0609K75F9P2R2,Ok,r06c09f09p02-ch2sk75fk1fl1.tiff,6,9,9,2,74,2,1,...,2E-06,0.135535598,266399.61,2021-04-19T21:14:19.757+01:00,640,706,40,1.1,0.2,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."
113398,0609K75F9P3R1,Ok,r06c09f09p03-ch1sk75fk1fl1.tiff,6,9,9,3,74,1,1,...,4E-06,0.135537595,266399.61,2021-04-19T21:14:20.037+01:00,488,522,40,1.1,0.1,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."


### View assay layout and mask information (optional)

The Opera Phenix acquires many time lapse series from a range of positions. The first step is to inspect the image metadata, presented in the form of an `Assaylayout/experiment_ID.xml` file, to show which positions correspond to which experimental assays.

In [19]:
metadata_path = glob.glob(os.path.join(base_dir, 'acquisition/Assaylayout/*.xml'))[0]
assay_layout = dataio.read_harmony_metadata(metadata_path, assay_layout=True,replicate_number=False)# mask_exist=True,  image_dir = image_dir, image_metadata = metadata)
assay_layout

Reading metadata XML file...
Extracting metadata complete!


Strain Compound Concentration ConcentrationEC
Row Column                                              
3   4         RD1     CTRL             0             EC0
    5          WT     CTRL             0             EC0
    6          WT      PZA            60            EC50
    7          WT      RIF           0.1            EC50
    8          WT      INH          0.04            EC50
    9          WT      BDQ          0.02            EC50
4   4         RD1     CTRL             0             EC0
    5          WT     CTRL             0             EC0
    6          WT      PZA            60            EC50
    7          WT      RIF           0.1            EC50
    8          WT      INH          0.04            EC50
    9          WT      BDQ          0.02            EC50
5   4         RD1     CTRL             0             EC0
    5          WT     CTRL             0             EC0
    6          WT      PZA           400            EC99
    7          WT      RIF             2            EC99
    8          WT      INH             2            EC99
    9          WT      BDQ           2.5            EC99
6   4         RD1     CTRL             0             EC0
    5          WT     CTRL             0             EC0
    6          WT      PZA           400            EC99
    7          WT      RIF             2            EC99
    8          WT      INH             2            EC99
    9          WT      BDQ           2.5            EC99

### Load using Zarr

In [20]:
acq_ID = (6, 4)

In [5]:
image_dir = os.path.join(base_dir, f'acquisition/zarr/{acq_ID}.zarr')

zarr_group = zarr.open(image_dir, mode='r')

In [7]:
%%time
images = zarr_group.images[:,:,:,...]
images.shape

CPU times: user 35.4 s, sys: 6min 18s, total: 6min 53s
Wall time: 6min 31s


(154, 2, 3, 6048, 6048)

In [6]:
images_max_proj = np.max(zarr_group.images, axis = 2)

In [8]:
images_max_proj.shape

(154, 2, 6048, 6048)

In [7]:
images.shape

NameError: name 'images' is not defined

#### Load downscaled version from mp4 videos

In [8]:
mp4_path = glob.glob(f'/mnt/SYNO/videos/macrohet_videos/{expt_ID}/{expt_ID}_{acq_ID}*')[0]

# Launch napari

In [8]:
viewer = napari.Viewer(title = f'{expt_ID, acq_ID}, max_proj')


viewer.add_image(images_max_proj,
                 channel_axis = 1, 
                 colormap=['magenta', 'green'],
                 blending = 'additive', 
                 contrast_limits=[[350, 1000],[0, 7000]]
                )

[<Image layer 'Image' at 0x7f22a9bee5e0>,
 <Image layer 'Image [1]' at 0x7f248b71ed00>]

In [9]:
with btrack.io.HDF5FileHandler(os.path.join(f'/mnt/SYNO/macrohet_syno/data/{expt_ID}/labels/cpv3/{acq_ID}.h5'), #macrohet_seg_model 
                                           'r', 
                                           obj_type='obj_type_1'
                                           ) as reader:
                segmentation = reader.segmentation
                tracks = reader.tracks

[INFO][2024/09/17 11:11:51 am] Opening HDF file: /mnt/SYNO/macrohet_syno/data/ND0003/labels/cpv3/(6, 4).h5...
INFO:btrack.io.hdf:Opening HDF file: /mnt/SYNO/macrohet_syno/data/ND0003/labels/cpv3/(6, 4).h5...
[INFO][2024/09/17 11:12:14 am] Loading segmentation (154, 6048, 6048)
INFO:btrack.io.hdf:Loading segmentation (154, 6048, 6048)
[INFO][2024/09/17 11:12:14 am] Loading tracks/obj_type_1
INFO:btrack.io.hdf:Loading tracks/obj_type_1
[INFO][2024/09/17 11:12:14 am] Loading LBEP/obj_type_1
INFO:btrack.io.hdf:Loading LBEP/obj_type_1
[INFO][2024/09/17 11:12:14 am] Loading objects/obj_type_1 (29627, 5) (29627 filtered: None)
INFO:btrack.io.hdf:Loading objects/obj_type_1 (29627, 5) (29627 filtered: None)
[INFO][2024/09/17 11:12:15 am] Closing HDF file: /mnt/SYNO/macrohet_syno/data/ND0003/labels/cpv3/(6, 4).h5
INFO:btrack.io.hdf:Closing HDF file: /mnt/SYNO/macrohet_syno/data/ND0003/labels/cpv3/(6, 4).h5


### Add tracks

In [10]:
filtered_tracks = [track for track in tracks if len(track) > 35]

In [27]:
recoloured_seg = btrack.utils.update_segmentation(segmentation, filtered_tracks, scale = (5.04, 5.04))

In [11]:
napari_tracks, _, _ = btrack.utils.tracks_to_napari(filtered_tracks, ndim = 2)

In [12]:
viewer.add_tracks(napari_tracks, scale=(1, 5.04, 5.04), name = 'filtered')

<Tracks layer 'filtered' at 0x7f229c1079a0>

### Add segmentation

In [13]:
viewer.add_labels(segmentation)
# viewer.add_labels(recoloured_seg)

<Labels layer 'segmentation' at 0x7f229c09b910>

In [ ]:
images_max_proj.shape

In [28]:
viewer.add_image(images_max_proj[-5],
                 channel_axis = 0, 
                 colormap=['green', 'magenta'],
                 blending = 'additive', 
                 contrast_limits=[[0, 2400],[0, 1000]]
                )

[<Image layer 'Image [4]' at 0x7fa481516460>,
 <Image layer 'Image [5]' at 0x7fa4823087f0>]

In [ ]:
viewer.camera.reset()
`

In [14]:
visualise.highlight_cell(126, viewer, filtered_tracks, scale_factor=5.02)

<Points layer 'cell 126' at 0x7f248be4fa60>

In [20]:
for layer in viewer.layers:
    viewer.layers[layer.name].scale = (10, 1, 1)

In [21]:
viewer.layers[2].name

'filtered'

In [22]:
viewer.layers[2].scale = (10,5.04, 5.04)